In [2]:
import cv2
import numpy as np
from math import *

undistorted_image = cv2.imread("000001.jpg")
undistorted_image = cv2.resize(undistorted_image, (undistorted_image.shape[1]*2, undistorted_image.shape[0]*2), interpolation=cv2.INTER_CUBIC)
undistorted_image.shape

(1800, 3200, 3)

In [3]:
def PointMap(x, y, r):
    ''' 
    将一个柱面图像上的点用等距模型映射到单位柱面
    input:
        x,y: 输入参数，畸变图像上的点的坐标
        theta_max 柱面镜头视场角
        r  制作的柱面图像半径
    output:
        new_x, new_y : 输出参数，单位球面上点的坐标
    '''
    f = r
    tx = f * tan(x / f)
    ty = y / cos(x / f)

    if x > 0:
        new_x = abs(tx)
    elif x < 0:
        new_x = - abs(tx)
    else:
        new_x = 0
    if y > 0:
        new_y = abs(ty)
    elif y < 0:
        new_y = - abs(ty)
    else:
        new_y = 0

    return new_x, new_y


In [4]:

def RectifyMap(r, ori_height, ori_width):
    '''
    生成从原图像到鱼眼图像的坐标的映射矩阵mapx mapy  
    input:
	    r: 圆半径，鱼眼图像半径
    output:
        mapy, mapy: 映射关系
    '''
    height = ori_height   # 映射图像的宽度
    s = 720.0 / 480.0    # 图像高和宽的比例

    width = int(height * s)
    center_x = width / 2
    center_y = height / 2

    mapx = np.zeros((height, width), dtype=float)
    mapy = np.zeros((height, width), dtype=float)

    for i in range(height):
        for j in range(width):
            x = j - center_x
            y = center_y - i

            nx, ny = PointMap(x, y, r)
            mapx[i][j] = nx
            mapy[i][j] = ny
    
    return mapx, mapy

ori_height, ori_width = undistorted_image.shape[0:2]
mapx, mapy = RectifyMap(800, ori_height, ori_width)

In [6]:
def UndisImage(undistorted_image, mapx, mapy):
    '''
    矫正图像
    '''
    assert(mapx.shape == mapy.shape)
    height, width = undistorted_image.shape[0:2]
    channel = undistorted_image.shape[-1]
    cx = width / 2
    cy = height / 2

    distort_height, distort_width = mapx.shape
    center_x = distort_width / 2
    center_y = distort_height / 2

    distort_image = np.zeros((distort_height, distort_width, channel), dtype=undistorted_image.dtype)

    for i in range(distort_height):
        for j in range(distort_width):
            x = int(mapx[i][j] + cx)
            y = int(cy - mapy[i][j])

            if x - cx == -1 and cy - y == -1: #   鱼眼图圆形外不填充像素
                continue
            if x < 0 or x >= width or y < 0 or y >= height: # //若圆形区域内某位置，对应原图上超出范围，不填充像素，这是黑边的由来
                continue
            distort_image[i][j] = undistorted_image[y][x]

    return distort_image

distort_image = UndisImage(undistorted_image, mapx, mapy)
distort_image.shape

cv2.imwrite('cylindrical.jpg', distort_image)

True